# Basic usage of Langchain's PromptTemplate
Exploring what I can do with the basic prompt template.

"Prompt is the input to the model"

In [ ]:
%pip install langchain

After installing langchain into the virtual environment we can start playing around 

PromptTemplate schema:
```python
class PromptTemplate(
    *,
    name: str | None = None,
    input_variables: List[str],
    input_types: Dict[str, Any] = dict,
    output_parser: BaseOutputParser | None = None,
    partial_variables: Mapping[str, str | (() -> str)] = dict,
    template: str,
    template_format: Literal['f-string', 'jinja2'] = "f-string",
    validate_template: bool = False
)
``````


In [4]:
from langchain.prompts import PromptTemplate

### Simple

In [11]:

prompt = PromptTemplate.from_template("Why is the sky the color blue")
prompt.format()

'Why is the sky the color blue'

### With template and variables

In [10]:
prompt_template_string = "Why is the {material} the color {color}"

prompt_template = PromptTemplate.from_template(template=prompt_template_string)
prompt_template.format(material="sky", color="blue")

'Why is the sky the color blue'

### Chain and Runnable 
Many components, including Prompt implements the Runnable protocol. Which is a standard interface making it easier to define custom chains and invoke them in different ways. 

- Runnable https://api.python.langchain.com/en/stable/runnables/langchain_core.runnables.base.Runnable.html#langchain_core.runnables.base.Runnable

In [15]:
prompt_template_string = "Why is the {material} the color {color}"

prompt_template = PromptTemplate.from_template(template=prompt_template_string)
prompt_chain = prompt
prompt_chain.invoke({"material": "sky", "color": "blue"})

StringPromptValue(text='Why is the sky the color blue')

### Graphs
could be useful to be able to visualize how the prompt is constucted, especially when constucting more complex chains. Thats why we can install a tool to draw out the built in `.get_graph()` in ascii. The graph function lets us explore inputs and outputs and the relationship between the complonents in the chain.

In [ ]:
%pip install grandalf

In [26]:

prompt_template = PromptTemplate.from_template(template=prompt_template_string)
prompt_chain = prompt 
prompt_chain.get_graph().print_ascii()
# prompt_chain.invoke({"material": "sky", "color": "blue"})

    +-------------+      
    | PromptInput |      
    +-------------+      
            *            
            *            
            *            
   +----------------+    
   | PromptTemplate |    
   +----------------+    
            *            
            *            
            *            
+----------------------+ 
| PromptTemplateOutput | 
+----------------------+ 


### Composability
There are a few ways to compose the basic PromptTemplate. The basic ones are Join, Partials and Pipelines.

- Partials https://python.langchain.com/docs/modules/model_io/prompts/partial
- Pipeline https://python.langchain.com/docs/modules/model_io/prompts/pipeline

In [83]:
# Joining prompts is basically like joining strings

# Creating a prompt template that adds additional context
context = PromptTemplate.from_template( """You are a scientist studying colors.""")

# Creating a basic prompt template
prompt = PromptTemplate.from_template(
"""
You get asked the following question: {question} 
What is your response?
""")

# Composing the two templates
composed_prompt = context + prompt
# Adding validation input variables to the composed prompt
composed_prompt.input_variables = ["question"]

prompt_text = composed_prompt.format(question="Why is the sky blue")
print(prompt_text) # returns a string


'You are a scientist studying colors.\nYou get asked the following question: Why is the sky blue \nWhat is your response?\n'